In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import gc
import torch
from internal.utils import sfm_outlier_detection

In [ ]:
parsed_poses_file = os.path.expanduser("~/dataset/dbl/parsed_from_exif-sift_matching/parsed.npy")  # produced by dji2pose.ipynb
# [NOTE] Remember to run a `point_triangulator` if it is merged from multiple models (e.g. `hierarchical_mapper`, `model_merger`)
sparse_model_dir = os.path.expanduser("~/dataset/dbl/parsed_from_exif-sift_matching/sfm_partitions-merged/final-retriangulated")
output_name = "outliers_removed"

In [ ]:
colmap_images = sfm_outlier_detection.load(parsed_poses_file, sparse_model_dir)

In [ ]:
# tune the parameters untill it produces the expected results
saver = sfm_outlier_detection.filter(
    colmap_images,
    output_name=output_name,
    min_acceptable_error_limit=6.,
    max_acceptable_error_limit=64.,
    device="cuda",  # use CUDA to speedup calculation
)

In [ ]:
outliers_removed_model_dir = saver(
    update_3d_points=False,  # must be True if it is used for `model_merger`
)
outliers_removed_model_dir

In [ ]:
del saver
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# [Optional] Bundle adjustment
ba_output_path = "{}-bundle_adjusted".format(outliers_removed_model_dir)
os.makedirs(ba_output_path, exist_ok=True)
print(" \\\n    ".join([
    "colmap",
    "bundle_adjuster",
    "--input_path={}".format(outliers_removed_model_dir),
    "--output_path={}".format(ba_output_path),
    "--BundleAdjustment.use_gpu=1",
]))